In [48]:
from utils import *

In [2]:
def flood_fill(test_array,h_max=255):
    input_array = np.copy(test_array) 
    el = sp.ndimage.generate_binary_structure(2,2).astype(np.int)
    inside_mask = sp.ndimage.binary_erosion(~np.isnan(input_array), structure=el)
    output_array = np.copy(input_array)
    output_array[inside_mask]=h_max
    output_old_array = np.copy(input_array)
    output_old_array.fill(0)   
    el = sp.ndimage.generate_binary_structure(2,1).astype(np.int)
    while not np.array_equal(output_old_array, output_array):
        output_old_array = np.copy(output_array)
        output_array = np.maximum(input_array,sp.ndimage.grey_erosion(output_array, size=(3,3), footprint=el))
    return output_array

In [3]:
def bwareaopen(img, min_size, connectivity=8):
        """Remove small objects from binary image (approximation of 
        bwareaopen in Matlab for 2D images).
    
        Args:
            img: a binary image (dtype=uint8) to remove small objects from
            min_size: minimum size (in pixels) for an object to remain in the image
            connectivity: Pixel connectivity; either 4 (connected via edges) or 8 (connected via edges and corners).
    
        Returns:
            the binary image with small objects removed
        """
    
        # Find all connected components (called here "labels")
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(
            img, connectivity=connectivity)
        
        # check size of all connected components (area in pixels)
        for i in range(num_labels):
            label_size = stats[i, cv2.CC_STAT_AREA]
            
            # remove connected components smaller than min_size
            if label_size < min_size:
                img[labels == i] = 0
                
        return img

In [4]:
def get_faces(img):
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    
    #inverse of the avg values of the RBG
    mR = 1/(np.mean(np.mean(R)))
    mG = 1/(np.mean(np.mean(G)))
    mB = 1/(np.mean(np.mean(B)))
    # smallest avg value
    maxRGB = max(max(mR, mG), mB)
    
    #calculate the scaling factors
    mR = mR/maxRGB
    mG = mG/maxRGB
    mB = mB/maxRGB

    out = np.zeros(img.shape)
    out = np.array(out, dtype=np.uint8)
    #scale the values
    out[:,:,0]=R*mR
    out[:,:,1]=G*mG
    out[:,:,2]=B*mB

    #convert the RGB image to YCbCr
    img_ycbcr =  cv2.cvtColor(out, cv2.COLOR_BGR2YCR_CB)

    #3. Extracting each component
    Y= img_ycbcr[:,:,0]
    Cb= img_ycbcr[:,:,1]
    Cr= img_ycbcr[:,:,2]

    #4. perform multi-level tresholding
    r = []
    c = []
    for i in range(img_ycbcr.shape[0]):
        for j in range(img_ycbcr.shape[1]):
            if Cb[i,j]>=77 and Cb[i,j]<=127 and Cr[i,j]>=133 and Cr[i,j]<=193 :
                r.append(i)
                c.append(j)

    numind=len(r)
    bin=np.zeros((img.shape[0],img.shape[1]))
    bin = np.array(bin, dtype=np.uint8)

    for i in range(numind):
        bin[r[i],c[i]]=1
    bin = flood_fill(bin)
    bin=bwareaopen(bin,9000)
    R[bin == 0]=0
    G[bin == 0]=0
    B[bin == 0]=0

    out[:,:,0] = R
    out[:,:,1] = G
    out[:,:,2] = B

    out_gray = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
    out_gray[out_gray > 0] = 1
    bounding_boxes = find_contours(out_gray, 0.8)

    comps,labels= cv2.connectedComponents(out_gray, connectivity=8)

    arr = []
    for k in range(1,comps):
        xcoordinate, ycoordinate = np.where(labels==k)
        #print(ycoordinate)
        #print(xcoordinate)
        max_x = -9999999999
        min_x = 9999999999

        max_y = -999999999
        min_y = 99999999999
        for j in range(xcoordinate.shape[0]):
            ypoint = ycoordinate[j]
            xpoint = xcoordinate[j]
            if(xpoint > max_x):
                max_x = xpoint
            if(xpoint < min_x):
                min_x = xpoint
            if(ypoint > max_y):
                max_y = ypoint
            if(ypoint < min_y):
                min_y = ypoint
        
        arr.append([min_x, min_y, max_y-min_y, max_x-min_x])
    return arr


### Training the Model(s)

In [12]:
def readFiles(PATH, imgs, labels, label) -> None:
    files = os.listdir(PATH)
    num_images = len(files)
    idx = 0
    for f in files:
        if(idx >= 100):
            break
        idx += 1
        img = io.imread(os.path.join(PATH, f), as_gray=True)
        imgs.append(img)
        labels.append(label)
    print(f"Done reading label {label}")

Read Training dataset from directories and label it

In [19]:
imgs = []
labels = []
features = []
HAPPY_PATH = './train/happy'
SAD_PATH = './train/sad'
ANGRY_PATH = './train/angry'
DISGUSTED_PATH = './train/disgusted'
FEARFUL_PATH = './train/fearful'
NEUTRAL_PATH = './train/neutral'
SURPRISED_PATH = './train/surprised'
readFiles(HAPPY_PATH,       imgs, labels, label=1)
readFiles(SAD_PATH,         imgs, labels, label=2)
readFiles(ANGRY_PATH,       imgs, labels, label=3)
readFiles(DISGUSTED_PATH,   imgs, labels, label=4)
readFiles(FEARFUL_PATH,     imgs, labels, label=5)
readFiles(NEUTRAL_PATH,     imgs, labels, label=6)
readFiles(SURPRISED_PATH,   imgs, labels, label=7)
labels = np.array(labels)

Done reading label 1
Done reading label 2
Done reading label 3
Done reading label 4
Done reading label 5
Done reading label 6
Done reading label 7


#### Reading the test dataset

In [63]:
imgs_test = []
labels_test = []
features_test = []
HAPPY_PATH = './test/happy'
SAD_PATH = './test/sad'
ANGRY_PATH = './test/angry'
DISGUSTED_PATH = './test/disgusted'
FEARFUL_PATH = './test/fearful'
NEUTRAL_PATH = './test/neutral'
SURPRISED_PATH = './test/surprised'
# readFiles(HAPPY_PATH,       imgs_test, labels, label=1)
# readFiles(SAD_PATH,         imgs_test, labels, label=2)
# readFiles(ANGRY_PATH,       imgs_test, labels, label=3)
# readFiles(DISGUSTED_PATH,   imgs_test, labels, label=4)
# readFiles(FEARFUL_PATH,     imgs_test, labels, label=5)
# readFiles(NEUTRAL_PATH,     imgs_test, labels, label=6)
# readFiles(SURPRISED_PATH,   imgs_test, labels, label=7)
# labels_test = np.array(labels)

## Method 1:
 LPQ + PHOG + SVM(calssifier)

In [20]:
cnt=  0
for img in imgs:
    lpq = apply_LPQ(img, 5)
    lpq, _ = np.histogram(lpq, 256)
    phog = PHOG_Algorithm(img, 8, 3)
    feature: np.ndarray = np.concatenate((lpq, phog))
    features.append(feature)
    cnt += 1
    if(cnt % 50 == 0):
        print(cnt)
features = np.array(features)

50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [61]:
pca = PCA(n_components=300)
features_new = pca.fit_transform(features)
pickle.dump(pca, open('model_1_PCA.sav', 'wb'))

# Get the SVC classifier
clf = svm.SVC()
# Train the SVC with the training data (data points and labels)
model = clf.fit(features_new, labels)
pickle.dump(model, open("model1.sav", 'wb'))
# Predict the test samples
pred = clf.predict(features_new)
print(np.sum(labels == pred) / 700.0)

0.5028571428571429


## Method 2:
PHOG + SVM(calssifier)

In [31]:
cnt=  0
features_2 = []
for img in imgs:
    phog = PHOG_Algorithm(img, 8, 3)
    features_2.append(phog)
    cnt += 1
    if(cnt % 50 == 0):
        print(cnt)
features_2 = np.array(features_2)

50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [ ]:
cnt=  0
for img in imgs_test:
    phog = PHOG_Algorithm(img, 8, 3)
    features_test.append(phog)
    cnt += 1
    if(cnt % 50 == 0):
        print(cnt)
features_2 = np.array(features_2)

In [58]:
pca = PCA(n_components=300)
features_new = pca.fit_transform(features_2)
pickle.dump(pca, open('model_2_PCA.sav', 'wb'))
# Get the SVC classifier
clf = svm.SVC()
# Train the SVC with the training data (data points and labels)
model = clf.fit(features_new, labels)
pickle.dump(clf, open("model2.sav", 'wb'))
# pickle.dump(model, open("SVM.mod", 'wb'))
# Predict the test samples
new = pca.transform(features_2[0].reshape((1, -1)))
pred = clf.predict(features_new)
acc = np.sum(labels == pred) / len(labels)
print(f'Accuracy when using only PHOG: {acc}')


Accuracy when using only PHOG: 0.82


## Method 3:
LPQ + SVM(calssifier)

In [44]:
features_3 = []
cnt=  0
for img in imgs:
    lpq = apply_LPQ(img, 5)
    lpq, _ = np.histogram(lpq, 256)
    features_3.append(lpq)
    cnt += 1
    if(cnt % 50 == 0):
        print(cnt)
features_3 = np.array(features_3)

50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [59]:
pca = PCA(n_components=200)
features_new = pca.fit_transform(features_3)
pickle.dump(pca, open('model_3_PCA.sav', 'wb'))

# Get the SVC classifier
clf = svm.SVC()
# Train the SVC with the training data (data points and labels)
model = clf.fit(features_new, labels)
pickle.dump(clf, open("model3.sav", 'wb'))
# pickle.dump(model, open("SVM.mod", 'wb'))
# Predict the test samples
pred = clf.predict(features_new)
acc = np.sum(labels == pred) / len(labels)
print(f'Accuracy when using only LPQ: {acc}')


Accuracy when using only LPQ: 0.5014285714285714


## Method 4:
LPQ + PHOG +LMNN(calssifier)

In [51]:
# from metric_learn import LMNN
# pca = PCA(n_components=300)

# features_new = pca.fit_transform(features)

# lmnn = LMNN(k=5, learn_rate=1e-6)
# lmnn.fit(features_new, labels)

# pred = lmnn.(features_new)
# acc = np.sum(labels == pred) / len(labels)
# print(f'Accuracy when using LMNN + LPQ + PHOG: {acc}')